In [1]:
import sqlite3
import pandas as pd

In [10]:
# Define the path to the primary database
primary_db_path = r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\USA\us_company_info.db'

# Connect to the primary database and load the data
conn = sqlite3.connect(primary_db_path)
primary_df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
conn.close()

print(len(primary_df))

11837


In [11]:
conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\DE\de_company_info.db')
deutsche_boerse_df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
print(len(deutsche_boerse_df))
conn.close()

conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\UK\uk_company_info.db')
lse_df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
print(len(lse_df))
conn.close()


3553
1803


In [2]:
def create_merged_database(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS companies (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            company_sec_cik TEXT,
            company_name TEXT,
            company_isin TEXT,
            company_ticker TEXT,
            company_exchange TEXT,
            country TEXT,
            industry TEXT,
            sector TEXT,
            longBusinessSummary TEXT,
            embedding BLOB,
            UNIQUE(company_name, company_ticker, company_isin)
        )
    ''')
    conn.commit()
    conn.close()

#create_merged_database('merged_company_info.db')

In [3]:
def insert_data(df, conn):
    c = conn.cursor()
    for _, row in df.iterrows():
        c.execute('''
            INSERT OR IGNORE INTO companies (
                company_sec_cik, company_name, company_isin, company_ticker, company_exchange,
                country, industry, sector, longBusinessSummary, embedding
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['company_sec_cik'], row['company_name'], row['company_isin'], row['company_ticker'], 
            row['company_exchange'], row['country'], row['industry'], row['sector'], 
            row['longBusinessSummary'], row.get('embedding')
        ))
    conn.commit()

In [10]:
def load_data(db_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
    conn.close()
    return df

In [12]:
# Insert data into merged database
conn = sqlite3.connect("merged_company_info.db")
insert_data(primary_df, conn)
insert_data(deutsche_boerse_df, conn)
insert_data(lse_df, conn)
conn.close()

In [13]:
conn = sqlite3.connect("merged_company_info.db")
df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
print(len(df))
conn.close()

17191


In [5]:
# Create empty merged database
import sqlite3
import pandas as pd

def remove_duplicates(df):
    # Create a helper column to prioritize non-OTC exchanges
    df['is_otc'] = df['company_exchange'].apply(lambda x: 1 if x == 'OTC' else 0)
    
    # Sort by company_name, and the helper column
    df = df.sort_values(by=['company_name', 'is_otc'])
    
    # Drop duplicates, keeping the first occurrence (non-OTC will be first due to sorting)
    df = df.drop_duplicates(subset=['company_name'], keep='first')
    
    # Drop the helper column
    df = df.drop(columns=['is_otc'])
    
    return df

def insert_data(df, conn):
    c = conn.cursor()
    for _, row in df.iterrows():
        c.execute('''
            INSERT OR IGNORE INTO companies (
                company_sec_cik, company_name, company_isin, company_ticker, company_exchange,
                country, industry, sector, longBusinessSummary, embedding
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['company_sec_cik'], row['company_name'], row['company_isin'], row['company_ticker'], 
            row['company_exchange'], row['country'], row['industry'], row['sector'], 
            row['longBusinessSummary'], row.get('embedding')
        ))
    conn.commit()

def load_data(db_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
    conn.close()
    return df

def filter_invalid_names(df):
    # Remove entries where company_name contains more than 2 adjacent spaces, or any of ",", "-", "/"
    df = df[~df['company_name'].str.contains(r'\s{3,}')]
    df = df[~df['company_name'].str.contains(r'[,\-\/]')]
    return df

# Load data from various sources with the specified conditions
conditions = """
    longBusinessSummary != 'N/A' AND
    country != 'N/A' AND
    industry != 'N/A' AND
    sector != 'N/A'
"""

primary_df = pd.read_sql_query(f"SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL AND {conditions}", sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\USA\us_company_info.db'))
deutsche_boerse_df = pd.read_sql_query(f"SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL AND {conditions}", sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\DE\de_company_info.db'))
lse_df = pd.read_sql_query(f"SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL AND {conditions}", sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\UK\uk_company_info.db'))

# Combine all dataframes
combined_df = pd.concat([primary_df, deutsche_boerse_df, lse_df])

print(f"Total records before filtering: {len(combined_df)}")

# Filter out invalid company names
combined_df = filter_invalid_names(combined_df)

print(f"Total records after filtering invalid names: {len(combined_df)}")

# Remove duplicates within the combined dataframe, prioritizing non-OTC exchanges
combined_df = remove_duplicates(combined_df)

print(f"Total records after deduplication: {len(combined_df)}")

# Creating empty merged database 
create_merged_database("merged_company_deduplicated.db")

# Insert data into merged database
conn = sqlite3.connect("merged_company_deduplicated.db")
insert_data(combined_df, conn)
conn.close()

Total records before filtering: 12391
Total records after filtering invalid names: 8264
Total records after deduplication: 6198


In [2]:
import sqlite3
import json
from difflib import SequenceMatcher
from collections import defaultdict

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio() >= 0.9  # Adjust the threshold as needed

def find_duplicates(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Fetch company_name and company_ticker
    c.execute('SELECT company_name, company_ticker FROM companies')
    companies = c.fetchall()
    
    conn.close()
    
    duplicates = defaultdict(list)
    
    for i, (name1, ticker1) in enumerate(companies):
        for j, (name2, ticker2) in enumerate(companies):
            if i != j:
                if similar(name1, name2) or similar(ticker1, ticker2):
                    duplicates[(name1, ticker1)].append((name2, ticker2))
    
    # Remove duplicate entries (since we're comparing bidirectionally)
    unique_duplicates = {k: list(set(v)) for k, v in duplicates.items()}
    
    # Convert the dictionary with tuple keys to a dictionary with string keys
    unique_duplicates_str_keys = {f"{k[0]}|{k[1]}": v for k, v in unique_duplicates.items()}
    
    # Convert to JSON format
    duplicates_json = json.dumps(unique_duplicates_str_keys, indent=4)
    
    # Write to a JSON file
    with open('duplicates.json', 'w') as f:
        f.write(duplicates_json)

# Run the function with your database path
find_duplicates('merged_company_deduplicated.db')


In [8]:
import sqlite3

def print_apple_companies(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Fetch all companies where company_name contains 'Apple' (case-insensitive)
    c.execute('''
        SELECT * FROM companies 
        WHERE LOWER(company_name) LIKE '%apple%'
    ''')
    apple_companies = c.fetchall()
    
    conn.close()
    
    for company in apple_companies:
        print(company)

# Run the function with your database path
print_apple_companies('merged_company_deduplicated.db')


(277, 'N/A', 'APPLE INC.', 'US0378331005', 'APC.DE', 'GER', 'United States', 'Consumer Electronics', 'Technology', 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and maga

In [9]:
import sqlite3

def normalize_name(name):
    return name.lower()

def count_missing_fields(record):
    return sum(1 for field in record if field is None or field == 'N/A')

def deduplicate_database(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Fetch all companies
    c.execute('SELECT * FROM companies')
    companies = c.fetchall()
    
    # Normalize company names and identify duplicates
    company_dict = {}
    for company in companies:
        normalized_name = normalize_name(company[2])
        if normalized_name not in company_dict:
            company_dict[normalized_name] = []
        company_dict[normalized_name].append(company)
    
    # List to hold IDs of rows to delete
    rows_to_delete = []
    
    for normalized_name, entries in company_dict.items():
        if len(entries) > 1:
            # Apply criteria for keeping entries
            entries.sort(key=lambda x: (
                x[1] == 'N/A' or x[1] is None,  # Prefer non-N/A and non-null company_sec_cik
                count_missing_fields(x)          # Prefer fewer missing fields
            ))
            # Keep the first entry and mark others for deletion
            rows_to_delete.extend(entry[0] for entry in entries[1:])
            #rows_to_delete.extend(entry for entry in entries[1:])

    
    # Remove duplicates from the database
    if rows_to_delete:
        c.executemany('DELETE FROM companies WHERE id = ?', [(id,) for id in rows_to_delete])
        conn.commit()
    
    conn.close()
    
    print(f"Deleted {len(rows_to_delete)} duplicate rows.")

# Run the function with your database path
deduplicate_database('merged_company_deduplicated.db')


Deleted 18 duplicate rows.


In [2]:
## Readding the entries that were filtered out by company name 

import sqlite3
import pandas as pd

def remove_duplicates(df):
    df['is_otc'] = df['company_exchange'].apply(lambda x: 1 if x == 'OTC' else 0)
    df = df.sort_values(by=['company_name', 'is_otc'])
    df = df.drop_duplicates(subset=['company_name'], keep='first')
    df = df.drop(columns=['is_otc'])
    return df

def insert_data(df, conn):
    c = conn.cursor()
    for _, row in df.iterrows():
        c.execute('''
            INSERT OR IGNORE INTO companies (
                company_sec_cik, company_name, company_isin, company_ticker, company_exchange,
                country, industry, sector, longBusinessSummary, embedding
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['company_sec_cik'], row['company_name'], row['company_isin'], row['company_ticker'], 
            row['company_exchange'], row['country'], row['industry'], row['sector'], 
            row['longBusinessSummary'], row.get('embedding')
        ))
    conn.commit()

def load_data(db_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
    conn.close()
    return df

def filter_invalid_names(df):
    invalid_names_df = df[df['company_name'].str.contains(r'\s{3,}|[,\-\/]')]
    valid_names_df = df[~df['company_name'].str.contains(r'\s{3,}|[,\-\/]')]
    return valid_names_df, invalid_names_df

def create_merged_database(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS companies (
            company_sec_cik TEXT,
            company_name TEXT,
            company_isin TEXT,
            company_ticker TEXT,
            company_exchange TEXT,
            country TEXT,
            industry TEXT,
            sector TEXT,
            longBusinessSummary TEXT,
            embedding BLOB
        )
    ''')
    conn.commit()
    conn.close()

# Load data from various sources with the specified conditions
conditions = """
    longBusinessSummary != 'N/A' AND
    country != 'N/A' AND
    industry != 'N/A' AND
    sector != 'N/A'
"""

primary_df = pd.read_sql_query(f"SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL AND {conditions}", sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\USA\us_company_info.db'))
deutsche_boerse_df = pd.read_sql_query(f"SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL AND {conditions}", sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\DE\de_company_info.db'))
lse_df = pd.read_sql_query(f"SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL AND {conditions}", sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\UK\uk_company_info.db'))

# Combine all dataframes
combined_df = pd.concat([primary_df, deutsche_boerse_df, lse_df])

print(f"Total records before filtering: {len(combined_df)}")

# Filter out invalid company names
valid_names_df, invalid_names_df = filter_invalid_names(combined_df)

print(invalid_names_df.head())

# Insert previously filtered invalid names into merged database
conn = sqlite3.connect("merged_company_deduplicated_readd_invalid_names.db")
insert_data(invalid_names_df, conn)
conn.close()

Total records before filtering: 12391
    id company_sec_cik                company_name company_ticker   
5    6         1326801        Meta Platforms, Inc.           META  \
8    9         1318605                 Tesla, Inc.           TSLA   
19  21          909832  COSTCO WHOLESALE CORP /NEW           COST   
23  25          354950            HOME DEPOT, INC.             HD   
24  26           70858   BANK OF AMERICA CORP /DE/            BAC   

   company_exchange        country                        industry   
5            Nasdaq  United States  Internet Content & Information  \
8            Nasdaq  United States              Auto Manufacturers   
19           Nasdaq  United States                 Discount Stores   
23             NYSE  United States         Home Improvement Retail   
24             NYSE  United States             Banks - Diversified   

                    sector                                longBusinessSummary   
5   Communication Services  Meta Platforms, I